In [ ]:
# 🎯 Test Loss: 0.2025 | Test Accuracy: 0.9302
# 🎯 Test Loss: 0.2940 | Test Accuracy: 0.8770
# 🎯 Test Loss: 0.2525 | Test Accuracy: 0.9151
# 🎯 Test Loss: 0.3163 | Test Accuracy: 0.8818

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir = "/root/autodl-fs/isic19_20_split/train"
val_dir   = "/root/autodl-fs/isic19_20_split/val"
test_dir  = "/root/autodl-fs/isic19_20_split/test"
ckpt_path     = "/root/autodl-fs/best_mobilenetv3.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-3  # MobileNet 通常用较大的学习率
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 使用 MobileNetV3 的官方预处理 —— #
weights = MobileNet_V3_Large_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. MobileNetV3 模型定义 —— #
model = mobilenet_v3_large(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 测试集评估 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:17<00:00, 1.30MB/s]


Epoch 1/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2742, Acc: 0.8966 | Val Loss: 0.2087, Acc: 0.9167
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2095, Acc: 0.9174 | Val Loss: 0.3804, Acc: 0.8981


...

[Epoch 10] Train Loss: 0.0515, Acc: 0.9808 | Val Loss: 0.2998, Acc: 0.9266


/tmp/ipykernel_78762/3485870107.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2025 | Test Accuracy: 0.9302

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_78762/3485870107.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2067 | Test Accuracy: 0.9109
📁 Epoch 02 | Test Loss: 0.3402 | Test Accuracy: 0.9034
📁 Epoch 03 | Test Loss: 0.2025 | Test Accuracy: 0.9302
📁 Epoch 04 | Test Loss: 0.1978 | Test Accuracy: 0.9284
📁 Epoch 05 | Test Loss: 0.2212 | Test Accuracy: 0.9185
📁 Epoch 06 | Test Loss: 0.1853 | Test Accuracy: 0.9249
📁 Epoch 07 | Test Loss: 0.2107 | Test Accuracy: 0.9139
📁 Epoch 08 | Test Loss: 0.2627 | Test Accuracy: 0.8807
📁 Epoch 09 | Test Loss: 0.2870 | Test Accuracy: 0.9086
📁 Epoch 10 | Test Loss: 0.3089 | Test Accuracy: 0.9191


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir = "/root/autodl-fs/processed/processed/train"
val_dir   = "/root/autodl-fs/processed/processed/val"
test_dir  = "/root/autodl-fs/processed/processed/test"
ckpt_path     = "/root/autodl-fs/best_mobilenetv3.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-3  # MobileNet 通常用较大的学习率
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 使用 MobileNetV3 的官方预处理 —— #
weights = MobileNet_V3_Large_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. MobileNetV3 模型定义 —— #
model = mobilenet_v3_large(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 测试集评估 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2550, Acc: 0.9020 | Val Loss: 2.0279, Acc: 0.7659
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2006, Acc: 0.9207 | Val Loss: 0.1984, Acc: 0.9282
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/287 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.1783, Acc: 0.9322 | Val Loss: 0.1794, Acc: 0.9319


/tmp/ipykernel_78762/894820141.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2493 | Test Accuracy: 0.9019

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_78762/894820141.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.5278 | Test Accuracy: 0.8536
📁 Epoch 02 | Test Loss: 0.2966 | Test Accuracy: 0.8858
📁 Epoch 03 | Test Loss: 0.3168 | Test Accuracy: 0.8726
📁 Epoch 04 | Test Loss: 0.2493 | Test Accuracy: 0.9019
📁 Epoch 05 | Test Loss: 0.2775 | Test Accuracy: 0.9004
📁 Epoch 06 | Test Loss: 0.3817 | Test Accuracy: 0.8653
📁 Epoch 07 | Test Loss: 133.3426 | Test Accuracy: 0.3075
📁 Epoch 08 | Test Loss: 0.4779 | Test Accuracy: 0.8551
📁 Epoch 09 | Test Loss: 0.2708 | Test Accuracy: 0.8960
📁 Epoch 10 | Test Loss: 0.2940 | Test Accuracy: 0.8770


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir = "/root/autodl-fs/generate/train"
val_dir   = "/root/autodl-fs/generate/val"
test_dir  = "/root/autodl-fs/generate/test"
ckpt_path     = "/root/autodl-fs/best_mobilenetv3.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-3  # MobileNet 通常用较大的学习率
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 使用 MobileNetV3 的官方预处理 —— #
weights = MobileNet_V3_Large_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. MobileNetV3 模型定义 —— #
model = mobilenet_v3_large(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 测试集评估 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/286 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2599, Acc: 0.8986 | Val Loss: 0.2102, Acc: 0.9189
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/286 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2079, Acc: 0.9198 | Val Loss: 0.3265, Acc: 0.8693


Epoch 3/10:   0%|          | 0/286 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0483, Acc: 0.9825 | Val Loss: 0.6662, Acc: 0.8712


/tmp/ipykernel_78762/3926138629.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2741 | Test Accuracy: 0.9048

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_78762/3926138629.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2525 | Test Accuracy: 0.9151
📁 Epoch 02 | Test Loss: 0.3411 | Test Accuracy: 0.8594
📁 Epoch 03 | Test Loss: 0.3090 | Test Accuracy: 0.8755
📁 Epoch 04 | Test Loss: 0.2741 | Test Accuracy: 0.9048
📁 Epoch 05 | Test Loss: 0.2562 | Test Accuracy: 0.9136
📁 Epoch 06 | Test Loss: 0.2957 | Test Accuracy: 0.9034
📁 Epoch 07 | Test Loss: 0.3783 | Test Accuracy: 0.9019
📁 Epoch 08 | Test Loss: 0.3786 | Test Accuracy: 0.8741
📁 Epoch 09 | Test Loss: 0.2833 | Test Accuracy: 0.9048
📁 Epoch 10 | Test Loss: 0.4722 | Test Accuracy: 0.8712


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from tqdm.notebook import tqdm

# —— 1. 忽略隐藏目录 —— #
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# —— 2. 配置 —— #
train_dir = "/root/autodl-fs/generate_twice/train"
val_dir   = "/root/autodl-fs/generate_twice/val"
test_dir  = "/root/autodl-fs/generate_twice/test"
ckpt_path     = "/root/autodl-fs/best_mobilenetv3.pth"
batch_size    = 32
num_epochs    = 10
learning_rate = 1e-3  # MobileNet 通常用较大的学习率
device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# —— 3. 使用 MobileNetV3 的官方预处理 —— #
weights = MobileNet_V3_Large_Weights.DEFAULT
transform = weights.transforms()

# —— 4. 加载数据 —— #
train_dataset = FilteredImageFolder(root=train_dir, transform=transform)
test_dataset  = FilteredImageFolder(root=test_dir,  transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(root=val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=4)

# —— 5. MobileNetV3 模型定义 —— #
model = mobilenet_v3_large(weights=weights)
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# —— 6. 训练 + 验证 + 保存最佳模型 —— #
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    # 验证
    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    # 每个 epoch 保存一次模型
    epoch_ckpt_path = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# —— 7. 测试集评估 —— #
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# —— 8. 所有 epoch 的模型推理 —— #
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []

for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_mobilenetv3/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()

    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.3412, Acc: 0.8550 | Val Loss: 0.3733, Acc: 0.8747
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2834, Acc: 0.8819 | Val Loss: 0.3441, Acc: 0.8647


Epoch 3/10:   0%|          | 0/283 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.1037, Acc: 0.9616 | Val Loss: 0.3616, Acc: 0.8866


/tmp/ipykernel_78762/1951740429.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.3163 | Test Accuracy: 0.8818

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_78762/1951740429.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.5172 | Test Accuracy: 0.8380
📁 Epoch 02 | Test Loss: 0.3285 | Test Accuracy: 0.8613
📁 Epoch 03 | Test Loss: 0.3060 | Test Accuracy: 0.8803
📁 Epoch 04 | Test Loss: 0.3251 | Test Accuracy: 0.8818
📁 Epoch 05 | Test Loss: 0.4496 | Test Accuracy: 0.8117
📁 Epoch 06 | Test Loss: 0.6848 | Test Accuracy: 0.7124
📁 Epoch 07 | Test Loss: 0.3163 | Test Accuracy: 0.8818
📁 Epoch 08 | Test Loss: 0.4530 | Test Accuracy: 0.8467
📁 Epoch 09 | Test Loss: 0.5746 | Test Accuracy: 0.8847
📁 Epoch 10 | Test Loss: 0.4334 | Test Accuracy: 0.8350
